# Multi Environment Config Management - SSM Backend

This is a follow up of [Multi Environment Config Management](https://github.com/MacHu-GWU/config_patterns-project/blob/main/example/multi_env_json/multi_environment_config.ipynb). In this article, we will introduce using [AWS System Manager Parameter Store](https://docs.aws.amazon.com/systems-manager/latest/userguide/systems-manager-parameter-store.html) as the backend to manage your multi-environment configurations.

We have prepared three versions of the config (v1, v2, v3) for testing purposes. Now, let's take a preview of the config data for each version.

In [1]:
import json
from rich import print as rprint

def jprint(data: dict):
    rprint(json.dumps(data, indent=4))

In [2]:
from confg_loader import (
    path_config_v1,
    path_config_secret_v1,
    path_config_v2,
    path_config_secret_v2,
    path_config_v3,
    path_config_secret_v3,
)

print("------ Version 1 ------")
rprint(path_config_v1.read_text())
rprint(path_config_secret_v1.read_text())

print("------ Version 2: ------")
rprint(path_config_v2.read_text())
rprint(path_config_secret_v2.read_text())

print("------ Version 3: ------")
rprint(path_config_v3.read_text())
rprint(path_config_secret_v3.read_text())

------ Version 1 ------


# content of config.json
{
    # this config file support comments, you can put documentation in the config file
    "_shared": {
        "*.project_name": "my_project"
    },
    "dev": {
        "username": "dev.user.v1"
    },
    "prod": {
        "username": "prod.user.v1"
    }
}

# content of secret_config.json
{
    # this config file support comments, you can put documentation in the config file
    "_shared": {
    },
    "dev": {
        "password": "dev.password"
    },
    "prod": {
        "password": "prod.password"
    }
}

------ Version 2: ------


# content of config.json
{
    # this config file support comments, you can put documentation in the config file
    "_shared": {
        "*.project_name": "my_project"
    },
    "dev": {
        "username": "dev.user.v2"
    },
    "prod": {
        "username": "prod.user.v2"
    }
}

# content of secret_config.json
{
    # this config file support comments, you can put documentation in the config file
    "_shared": {
    },
    "dev": {
        "password": "dev.password"
    },
    "prod": {
        "password": "prod.password"
    }
}

------ Version 3: ------


# content of config.json
{
    # this config file support comments, you can put documentation in the config file
    "_shared": {
        "*.project_name": "my_project"
    },
    "dev": {
        "username": "dev.user.v3"
    },
    "prod": {
        "username": "prod.user.v3"
    }
}

# content of secret_config.json
{
    # this config file support comments, you can put documentation in the config file
    "_shared": {
    },
    "dev": {
        "password": "dev.password"
    },
    "prod": {
        "password": "prod.password"
    }
}

Similar to what we have done in [Multi Environment Config Management](https://github.com/MacHu-GWU/config_patterns-project/blob/main/example/multi_env_json/multi_environment_config.ipynb), we declared the config data model as below.

In [3]:
import os
# content of config_define.py
# -*- coding: utf-8 -*-

import typing as T
import os
import dataclasses

from config_patterns.patterns.multi_env_json.api import (
    BaseEnvEnum, # the base class of the environment name enum class
    BaseEnv, # the base class of the per environment object
    BaseConfig, # the base class of the all-in-one config object
)


class EnvEnum(BaseEnvEnum):
    dev = "dev" # development
    prod = "prod" # production


@dataclasses.dataclass
class Env(BaseEnv):
    username: T.Optional[str] = dataclasses.field(default=None)
    password: T.Optional[str] = dataclasses.field(default=None)

    @classmethod
    def from_dict(cls, data: dict):
        """
        This method defines how to create an instance of this class from a dict.

        Example:

            >>> Env.from_dict({"username": "user1", "password": "pass1"})
        """
        return cls(**data)

    @property
    def login_info(self) -> str:
        """
        This is a sample derived attribute.
        """
        return f"Hello {self.username}, please enter your password: "


@dataclasses.dataclass
class Config(BaseConfig):
    @property
    def dev(self) -> Env:
        """
        A shortcut to get the dev environment config object.
        """
        return self.get_env(EnvEnum.dev)

    @property
    def prod(self) -> Env:
        """
        A shortcut to get the dev environment config object.
        """
        return self.get_env(EnvEnum.prod)

    @classmethod
    def get_current_env(cls) -> str:
        """
        You may want a smarter way to determine the current environment.
        For example, you may define the local laptop is ``dev``, and the
        virtual machine is ``prod``.
        """
        if "IS_VM" in os.environ:
            return EnvEnum.prod.value
        else:
            return EnvEnum.dev.value

    @property
    def env(self) -> Env:
        """
        This is a shortcut to get the current environment object.
        """
        return self.get_env(self.get_current_env())

In this tutorial, we utilize [moto](https://docs.getmoto.org/en/latest/docs/getting_started.html) to mock AWS services. Therefore, you don't need to set up a real AWS account and can concentrate solely on understanding the concepts.

In [4]:
import moto
from boto_session_manager import BotoSesManager
from s3pathlib import S3Path

# mock related AWS services
mock_s3 = moto.mock_s3()
mock_sts = moto.mock_sts()
mock_s3.start()
mock_sts.start()

# create a boto session manager object 
bsm = BotoSesManager(region_name="us-east-1")
# define the bucket to store config files
bucket = "my-bucket"
# create the bucket first
rprint(bsm.s3_client.create_bucket(Bucket=bucket))

{
    'ResponseMetadata': {
        'RequestId': 'L4vwfOYc0Mb1pwgXDUlt1yHiuDcbdVyEyF1JlQvvVQgVZMHa5odl',
        'HTTPStatusCode': 200,
        'HTTPHeaders': {'x-amzn-requestid': 'L4vwfOYc0Mb1pwgXDUlt1yHiuDcbdVyEyF1JlQvvVQgVZMHa5odl'},
        'RetryAttempts': 0
    }
}

## S3 Backend Without Version Enabled (Default)

### Read the Config Object from Local JSON File

First, we read the config version 1 from local JSON file.

In [5]:
config_v1 = Config.read(
    env_class=Env,
    env_enum_class=EnvEnum,
    path_config=path_config_v1,
    path_secret_config=path_config_secret_v1,
)
rprint(config_v1)

Config(
    data={
        '_shared': {'*.project_name': 'my_project'},
        'dev': {'username': 'dev.user.v1'},
        'prod': {'username': 'prod.user.v1'}
    },
    secret_data={'_shared': {}, 'dev': {'password': 'dev.password'}, 'prod': {'password': 'prod.password'}},
    Env=<class '__main__.Env'>,
    EnvEnum=<enum 'EnvEnum'>,
    version='local',
    _applied_data={
        'dev': {'username': 'dev.user.v1', 'project_name': 'my_project'},
        'prod': {'username': 'prod.user.v1', 'project_name': 'my_project'}
    },
    _applied_secret_data={'dev': {'password': 'dev.password'}, 'prod': {'password': 'prod.password'}},
    _merged={
        'dev': {'username': 'dev.user.v1', 'project_name': 'my_project', 'password': 'dev.password'},
        'prod': {'username': 'prod.user.v1', 'project_name': 'my_project', 'password': 'prod.password'}
    }
)

The config object has two built-in attributes ``project_name`` and ``env_name``.

In [6]:
print(f"project_name: {config_v1.project_name}")
print(f"dev env_name: {config_v1.dev.env_name}")
print(f"prod env_name: {config_v1.prod.env_name}")

project_name: my_project
dev env_name: dev
prod env_name: prod


The config object also has a built-in derived attribute ``parameter_name``. It is the normalized name (convert to lower case and snake case (underscore only)) for config deployment resources name of your backend.

In [7]:
print(f"parameter_name: {config_v1.parameter_name}")

parameter_name: my_project


### Deploy Config to S3 Backend

Now, you can use the ``Config.deploy()`` method to deploy the config object to S3. It creates an all-in-one deployment containing all environment data as a backup for disaster recovery or compliance requirements. Additionally, it creates individual per-environment deployments, ensuring that each environment can only access its specific configuration data and cannot access the configuration of other environments.

In [8]:
s3folder_config = f"s3://{bucket}/my-project/"
deployment_list = config_v1.deploy(
    bsm=bsm, 
    s3folder_config=s3folder_config,
    # these two arguments are optional
    tags={"project_name": config_v1.project_name},
    verbose=True,
)

+----- ⏱ 🟢 Start 'deploy config file to S3' -----------------------------------+
| 
| 🚀️ deploy config file/files at s3://my-bucket/my-project/my_project/my_project-latest.json ...
| preview at: https://console.aws.amazon.com/s3/object/my-bucket?prefix=my-project/my_project/my_project-latest.json
| done!
| 
+----- ⏰ 🟢 End 'deploy config file to S3', elapsed = 0.04 sec -----------------+
+----- ⏱ 🟢 Start 'deploy config file to S3' -----------------------------------+
| 
| 🚀️ deploy config file/files at s3://my-bucket/my-project/my_project-dev/my_project-dev-latest.json ...
| preview at: https://console.aws.amazon.com/s3/object/my-bucket?prefix=my-project/my_project-dev/my_project-dev-latest.json
| done!
| 
+----- ⏰ 🟢 End 'deploy config file to S3', elapsed = 0.01 sec -----------------+
+----- ⏱ 🟢 Start 'deploy config file to S3' -----------------------------------+
| 
| 🚀️ deploy config file/files at s3://my-bucket/my-project/my_project-prod/my_project-prod-latest.json ...
| preview at:

Now, let's see what happens in AWS S3. In this example, six S3 objects are created. The file with suffix ``*-latest.json`` is the latest version of the config, and the file with suffix ``*-000001.json`` is a copy of the config object at specific version. Version is an auto-incremental version number from 1, 2, 3, .... Since we don't have S3 versioning enabled, so ``config_patterns`` manages the auto-incremental for you and keep a backup for all historical versions.

- ``s3://my-bucket/my-project/my_project/my_project-latest.json``
- ``s3://my-bucket/my-project/my_project/my_project-000001.json``
- ``s3://my-bucket/my-project/my_project-dev/my_project-dev-latest.json``
- ``s3://my-bucket/my-project/my_project-dev/my_project-dev-000001.json``
- ``s3://my-bucket/my-project/my_project-prod/my_project-prod-latest.json``
- ``s3://my-bucket/my-project/my_project-prod/my_project-prod-000001.json``

In [9]:
rprint(S3Path(s3folder_config).iter_objects(bsm=bsm).all())

[
    S3Path('s3://my-bucket/my-project/my_project-dev/my_project-dev-000001.json'),
    S3Path('s3://my-bucket/my-project/my_project-dev/my_project-dev-latest.json'),
    S3Path('s3://my-bucket/my-project/my_project-prod/my_project-prod-000001.json'),
    S3Path('s3://my-bucket/my-project/my_project-prod/my_project-prod-latest.json'),
    S3Path('s3://my-bucket/my-project/my_project/my_project-000001.json'),
    S3Path('s3://my-bucket/my-project/my_project/my_project-latest.json')
]

The config version of the config data is stored in the S3 object metadata. It also has the sha256 of the config data for integration check.

In [10]:
s3path = S3Path("s3://my-bucket/my-project/my_project/my_project-latest.json")
rprint(s3path.read_text(bsm=bsm))
rprint(s3path.metadata)

{
    "data": {
        "_shared": {
            "*.project_name": "my_project"
        },
        "dev": {
            "username": "dev.user.v1"
        },
        "prod": {
            "username": "prod.user.v1"
        }
    },
    "secret_data": {
        "_shared": {},
        "dev": {
            "password": "dev.password"
        },
        "prod": {
            "password": "prod.password"
        }
    }
}

{'config_version': '1', 'config_sha256': 'e18bf9f25134c83ef26355ecd058c5bc89c03341bf95ba129c973cc0961149d0'}

If you are trying to deploy the same config data with no change, ``config_patterns`` library can automatically detect that and skip the deployment.

In [11]:
deployment_list = config_v1.deploy(
    bsm=bsm, 
    s3folder_config=s3folder_config,
    # these two arguments are optional
    tags={"project_name": config_v1.project_name},
    verbose=True,
)

+----- ⏱ 🟢 Start 'deploy config file to S3' -----------------------------------+
| 
| 🚀️ deploy config file/files at s3://my-bucket/my-project/my_project/my_project-latest.json ...
| preview at: https://console.aws.amazon.com/s3/object/my-bucket?prefix=my-project/my_project/my_project-latest.json
| config data is the same as existing one, do nothing.
| 
+----- ⏰ 🟢 End 'deploy config file to S3', elapsed = 0.01 sec -----------------+
+----- ⏱ 🟢 Start 'deploy config file to S3' -----------------------------------+
| 
| 🚀️ deploy config file/files at s3://my-bucket/my-project/my_project-dev/my_project-dev-latest.json ...
| preview at: https://console.aws.amazon.com/s3/object/my-bucket?prefix=my-project/my_project-dev/my_project-dev-latest.json
| config data is the same as existing one, do nothing.
| 
+----- ⏰ 🟢 End 'deploy config file to S3', elapsed = 0.00 sec -----------------+
+----- ⏱ 🟢 Start 'deploy config file to S3' -----------------------------------+
| 
| 🚀️ deploy config file/fi

### Read Config from S3 Backend

Now, you can use the Config.read() method to readthe config object from S3. If you want to read the all-in-one config object, then you could use ``parameter_name="${parameter_name}"``. If you want to read the config object of specific environment, you could use ``parameter="${parameter_name}-${env_name}"``.

In [12]:
config = Config.read(
    env_class=Env,
    env_enum_class=EnvEnum,
    bsm=bsm,
    parameter_name="my_project",
    s3folder_config=s3folder_config,
)
print("all in one config object:")
rprint(config)

all in one config object:


Config(
    data={
        '_shared': {'*.project_name': 'my_project'},
        'dev': {'username': 'dev.user.v1'},
        'prod': {'username': 'prod.user.v1'}
    },
    secret_data={'_shared': {}, 'dev': {'password': 'dev.password'}, 'prod': {'password': 'prod.password'}},
    Env=<class '__main__.Env'>,
    EnvEnum=<enum 'EnvEnum'>,
    version='1',
    _applied_data={
        'dev': {'username': 'dev.user.v1', 'project_name': 'my_project'},
        'prod': {'username': 'prod.user.v1', 'project_name': 'my_project'}
    },
    _applied_secret_data={'dev': {'password': 'dev.password'}, 'prod': {'password': 'prod.password'}},
    _merged={
        'dev': {'username': 'dev.user.v1', 'project_name': 'my_project', 'password': 'dev.password'},
        'prod': {'username': 'prod.user.v1', 'project_name': 'my_project', 'password': 'prod.password'}
    }
)

In [13]:
config = Config.read(
    env_class=Env,
    env_enum_class=EnvEnum,
    bsm=bsm,
    parameter_name="my_project-dev",
    s3folder_config=s3folder_config,
)
print("dev config object:")
rprint(config)

dev config object:


Config(
    data={'_shared': {'*.project_name': 'my_project'}, 'dev': {'username': 'dev.user.v1'}},
    secret_data={'_shared': {}, 'dev': {'password': 'dev.password'}},
    Env=<class '__main__.Env'>,
    EnvEnum=<enum 'EnvEnum'>,
    version='1',
    _applied_data={'dev': {'username': 'dev.user.v1', 'project_name': 'my_project'}},
    _applied_secret_data={'dev': {'password': 'dev.password'}},
    _merged={'dev': {'username': 'dev.user.v1', 'project_name': 'my_project', 'password': 'dev.password'}}
)

### Deploy a New Version of Config

When you deploy a new version of the config, it replaces the ``*-latest.json`` file and creates a backup of the new version.

In [14]:
config_v2 = Config.read(
    env_class=Env,
    env_enum_class=EnvEnum,
    path_config=path_config_v2,
    path_secret_config=path_config_secret_v2,
)
rprint(config_v2)

deployment_list = config_v2.deploy(
    bsm=bsm, 
    s3folder_config=s3folder_config,
    # these two arguments are optional
    tags={"project_name": config_v2.project_name},
    verbose=True,
)

Config(
    data={
        '_shared': {'*.project_name': 'my_project'},
        'dev': {'username': 'dev.user.v2'},
        'prod': {'username': 'prod.user.v2'}
    },
    secret_data={'_shared': {}, 'dev': {'password': 'dev.password'}, 'prod': {'password': 'prod.password'}},
    Env=<class '__main__.Env'>,
    EnvEnum=<enum 'EnvEnum'>,
    version='local',
    _applied_data={
        'dev': {'username': 'dev.user.v2', 'project_name': 'my_project'},
        'prod': {'username': 'prod.user.v2', 'project_name': 'my_project'}
    },
    _applied_secret_data={'dev': {'password': 'dev.password'}, 'prod': {'password': 'prod.password'}},
    _merged={
        'dev': {'username': 'dev.user.v2', 'project_name': 'my_project', 'password': 'dev.password'},
        'prod': {'username': 'prod.user.v2', 'project_name': 'my_project', 'password': 'prod.password'}
    }
)

+----- ⏱ 🟢 Start 'deploy config file to S3' -----------------------------------+
| 
| 🚀️ deploy config file/files at s3://my-bucket/my-project/my_project/my_project-latest.json ...
| preview at: https://console.aws.amazon.com/s3/object/my-bucket?prefix=my-project/my_project/my_project-latest.json
| done!
| 
+----- ⏰ 🟢 End 'deploy config file to S3', elapsed = 0.01 sec -----------------+
+----- ⏱ 🟢 Start 'deploy config file to S3' -----------------------------------+
| 
| 🚀️ deploy config file/files at s3://my-bucket/my-project/my_project-dev/my_project-dev-latest.json ...
| preview at: https://console.aws.amazon.com/s3/object/my-bucket?prefix=my-project/my_project-dev/my_project-dev-latest.json
| done!
| 
+----- ⏰ 🟢 End 'deploy config file to S3', elapsed = 0.01 sec -----------------+
+----- ⏱ 🟢 Start 'deploy config file to S3' -----------------------------------+
| 
| 🚀️ deploy config file/files at s3://my-bucket/my-project/my_project-prod/my_project-prod-latest.json ...
| preview at:

Let's check the status of the S3 bucket. In each environment, there are three S3 objects: v1, v2, and latest.

In [15]:
rprint(S3Path(s3folder_config).iter_objects(bsm=bsm).all())

[
    S3Path('s3://my-bucket/my-project/my_project-dev/my_project-dev-000001.json'),
    S3Path('s3://my-bucket/my-project/my_project-dev/my_project-dev-000002.json'),
    S3Path('s3://my-bucket/my-project/my_project-dev/my_project-dev-latest.json'),
    S3Path('s3://my-bucket/my-project/my_project-prod/my_project-prod-000001.json'),
    S3Path('s3://my-bucket/my-project/my_project-prod/my_project-prod-000002.json'),
    S3Path('s3://my-bucket/my-project/my_project-prod/my_project-prod-latest.json'),
    S3Path('s3://my-bucket/my-project/my_project/my_project-000001.json'),
    S3Path('s3://my-bucket/my-project/my_project/my_project-000002.json'),
    S3Path('s3://my-bucket/my-project/my_project/my_project-latest.json')
]

### Delete and Clean Up

Normally, it is not necessary to delete any config deployments (S3 objects) as they have low cost and are critical to your application. If you accidentally deploy malformed config data, there is no need to delete it. Instead, you can simply create a new deployment with the corrected configuration.

To clean up all config objects in all environments, including the historical versions, you can retrieve the all-in-one object and then use the ``config.delete()`` method. By default, when you delete the object, the ``*-latest.json`` file is removed, making the config deployment "invisible (unaccessible)" to your application. However, the backup of all historical versions is retained. If you explicitly set ``include_history=True``, then all historical versions will be deleted as well. This design is implemented to prevent accidental deletion of all configurations. If you only delete the ``*-latest.json`` file, you can always recover it from the latest version number.

In [16]:
deployment_list = config_v2.delete(
    bsm=bsm, 
    s3folder_config=s3folder_config,
)

+----- ⏱ 🟢 Start 'delete config file from S3' ---------------------------------+
| 
| 🗑️ delete config file/files at: s3://my-bucket/my-project/my_project/my_project-latest.json ...
| preview at: https://console.aws.amazon.com/s3/object/my-bucket?prefix=my-project/my_project/my_project-latest.json
| done!
| 
+----- ⏰ 🟢 End 'delete config file from S3', elapsed = 0.02 sec ---------------+
+----- ⏱ 🟢 Start 'delete config file from S3' ---------------------------------+
| 
| 🗑️ delete config file/files at: s3://my-bucket/my-project/my_project-dev/my_project-dev-latest.json ...
| preview at: https://console.aws.amazon.com/s3/object/my-bucket?prefix=my-project/my_project-dev/my_project-dev-latest.json
| done!
| 
+----- ⏰ 🟢 End 'delete config file from S3', elapsed = 0.00 sec ---------------+
+----- ⏱ 🟢 Start 'delete config file from S3' ---------------------------------+
| 
| 🗑️ delete config file/files at: s3://my-bucket/my-project/my_project-prod/my_project-prod-latest.json ...
| preview 

In [17]:
rprint(S3Path(s3folder_config).iter_objects(bsm=bsm).all())

[
    S3Path('s3://my-bucket/my-project/my_project-dev/my_project-dev-000001.json'),
    S3Path('s3://my-bucket/my-project/my_project-dev/my_project-dev-000002.json'),
    S3Path('s3://my-bucket/my-project/my_project-prod/my_project-prod-000001.json'),
    S3Path('s3://my-bucket/my-project/my_project-prod/my_project-prod-000002.json'),
    S3Path('s3://my-bucket/my-project/my_project/my_project-000001.json'),
    S3Path('s3://my-bucket/my-project/my_project/my_project-000002.json')
]

In [18]:
deployment_list = config_v2.delete(
    bsm=bsm, 
    s3folder_config=s3folder_config,
    include_history=True,
)

+----- ⏱ 🟢 Start 'delete config file from S3' ---------------------------------+
| 
| 🗑️ delete config file/files at: s3://my-bucket/my-project/my_project/ ...
| preview at: https://console.aws.amazon.com/s3/buckets/my-bucket?prefix=my-project/my_project/
| done!
| 
+----- ⏰ 🟢 End 'delete config file from S3', elapsed = 0.02 sec ---------------+
+----- ⏱ 🟢 Start 'delete config file from S3' ---------------------------------+
| 
| 🗑️ delete config file/files at: s3://my-bucket/my-project/my_project-dev/ ...
| preview at: https://console.aws.amazon.com/s3/buckets/my-bucket?prefix=my-project/my_project-dev/
| done!
| 
+----- ⏰ 🟢 End 'delete config file from S3', elapsed = 0.00 sec ---------------+
+----- ⏱ 🟢 Start 'delete config file from S3' ---------------------------------+
| 
| 🗑️ delete config file/files at: s3://my-bucket/my-project/my_project-prod/ ...
| preview at: https://console.aws.amazon.com/s3/buckets/my-bucket?prefix=my-project/my_project-prod/
| done!
| 
+----- ⏰ 🟢 End 'del

In [19]:
rprint(S3Path(s3folder_config).iter_objects(bsm=bsm).all())

[]

## S3 Backend With Version Enabled

[Versioning](https://docs.aws.amazon.com/AmazonS3/latest/userguide/Versioning.html) is an awesome feature in AWS S3. With versioning enabled, overwriting an existing S3 object is actually creating a new version of it. And the deletion is just about putting a delete marker on top of the latest version and made this S3 object "invisible". The historical version are always stored. As a result, the ``config_patterns`` library doesn't need to manage the ``*-latest.json`` and ``*-${version}.json`` anymore.

Firstly, let's create a new bucket and enable versioning.


In [20]:
# define the bucket to store config files
bucket_versioned = "my-bucket-versioned"
# create the bucket first
bsm.s3_client.create_bucket(Bucket=bucket_versioned)
_ = bsm.s3_client.put_bucket_versioning(
    Bucket=bucket_versioned,
    VersioningConfiguration={
        "Status": "Enabled"
    },
)

### Deploy Config to S3 Backend

Just like what we did before, we read the config object from local JSON file and deploy it to S3.

You don't need to perform any additional steps to inform the ``config_patterns`` library about the usage of a versioning-enabled S3 bucket. The library can automatically detect this configuration. However, if you enable and then disable "versioning" for the S3 bucket, the versioning status will change to "Suspended". In this case, the library cannot handle this situation and will raise an error. Therefore, you should not use a "Suspended" bucket as the backend for the library.

In [21]:
config_v1 = Config.read(
    env_class=Env,
    env_enum_class=EnvEnum,
    path_config=path_config_v1,
    path_secret_config=path_config_secret_v1,
)
rprint(config_v1)

Config(
    data={
        '_shared': {'*.project_name': 'my_project'},
        'dev': {'username': 'dev.user.v1'},
        'prod': {'username': 'prod.user.v1'}
    },
    secret_data={'_shared': {}, 'dev': {'password': 'dev.password'}, 'prod': {'password': 'prod.password'}},
    Env=<class '__main__.Env'>,
    EnvEnum=<enum 'EnvEnum'>,
    version='local',
    _applied_data={
        'dev': {'username': 'dev.user.v1', 'project_name': 'my_project'},
        'prod': {'username': 'prod.user.v1', 'project_name': 'my_project'}
    },
    _applied_secret_data={'dev': {'password': 'dev.password'}, 'prod': {'password': 'prod.password'}},
    _merged={
        'dev': {'username': 'dev.user.v1', 'project_name': 'my_project', 'password': 'dev.password'},
        'prod': {'username': 'prod.user.v1', 'project_name': 'my_project', 'password': 'prod.password'}
    }
)

In [22]:
s3folder_config = f"s3://{bucket_versioned}/my-project/"
deployment_list = config_v1.deploy(
    bsm=bsm, 
    s3folder_config=s3folder_config,
    # these two arguments are optional
    tags={"project_name": config_v1.project_name},
    verbose=True,
)

+----- ⏱ 🟢 Start 'deploy config file to S3' -----------------------------------+
| 
| 🚀️ deploy config file/files at s3://my-bucket-versioned/my-project/my_project.json ...
| preview at: https://console.aws.amazon.com/s3/object/my-bucket-versioned?prefix=my-project/my_project.json
| done!
| 
+----- ⏰ 🟢 End 'deploy config file to S3', elapsed = 0.01 sec -----------------+
+----- ⏱ 🟢 Start 'deploy config file to S3' -----------------------------------+
| 
| 🚀️ deploy config file/files at s3://my-bucket-versioned/my-project/my_project-dev.json ...
| preview at: https://console.aws.amazon.com/s3/object/my-bucket-versioned?prefix=my-project/my_project-dev.json
| done!
| 
+----- ⏰ 🟢 End 'deploy config file to S3', elapsed = 0.01 sec -----------------+
+----- ⏱ 🟢 Start 'deploy config file to S3' -----------------------------------+
| 
| 🚀️ deploy config file/files at s3://my-bucket-versioned/my-project/my_project-prod.json ...
| preview at: https://console.aws.amazon.com/s3/object/my-bucket-v

Now, let's see what happens in AWS S3. With versioning enabled, only three S3 objects are created. And the concept of a specific "latest" object is not applicable in this case. This is because S3 object versioning automatically manages that. Now the version of the config data becomes the S3 object version id.

- ``s3://my-bucket/my-project/my_project/my_project.json``
- ``s3://my-bucket/my-project/my_project/my_project-dev.json``
- ``s3://my-bucket/my-project/my_project/my_project-prod.json``

In [23]:
rprint(S3Path(s3folder_config).iter_objects(bsm=bsm).all())

[
    S3Path('s3://my-bucket-versioned/my-project/my_project-dev.json'),
    S3Path('s3://my-bucket-versioned/my-project/my_project-prod.json'),
    S3Path('s3://my-bucket-versioned/my-project/my_project.json')
]

You can access the version id of the latest config data via S3 API.

In [24]:
s3path = S3Path("s3://my-bucket-versioned/my-project/my_project.json")
print("s3 object content:")
rprint(s3path.read_text(bsm=bsm))
v1 = s3path.version_id
print(f"version_id: {v1}")
rprint(f"s3 object metadata: {s3path.metadata}")

s3 object content:


{
    "data": {
        "_shared": {
            "*.project_name": "my_project"
        },
        "dev": {
            "username": "dev.user.v1"
        },
        "prod": {
            "username": "prod.user.v1"
        }
    },
    "secret_data": {
        "_shared": {},
        "dev": {
            "password": "dev.password"
        },
        "prod": {
            "password": "prod.password"
        }
    }
}

version_id: cc4c8cf1-0e3f-4f41-9925-090769850fb6


s3 object metadata: {'config_sha256': 'e18bf9f25134c83ef26355ecd058c5bc89c03341bf95ba129c973cc0961149d0'}

If you are trying to deploy the same config data with no change, config_patterns library can automatically detect that and skip the deployment.

In [25]:
deployment_list = config_v1.deploy(
    bsm=bsm, 
    s3folder_config=s3folder_config,
    # these two arguments are optional
    tags={"project_name": config_v1.project_name},
    verbose=True,
)

+----- ⏱ 🟢 Start 'deploy config file to S3' -----------------------------------+
| 
| 🚀️ deploy config file/files at s3://my-bucket-versioned/my-project/my_project.json ...
| preview at: https://console.aws.amazon.com/s3/object/my-bucket-versioned?prefix=my-project/my_project.json
| config data is the same as existing one, do nothing.
| 
+----- ⏰ 🟢 End 'deploy config file to S3', elapsed = 0.01 sec -----------------+
+----- ⏱ 🟢 Start 'deploy config file to S3' -----------------------------------+
| 
| 🚀️ deploy config file/files at s3://my-bucket-versioned/my-project/my_project-dev.json ...
| preview at: https://console.aws.amazon.com/s3/object/my-bucket-versioned?prefix=my-project/my_project-dev.json
| config data is the same as existing one, do nothing.
| 
+----- ⏰ 🟢 End 'deploy config file to S3', elapsed = 0.00 sec -----------------+
+----- ⏱ 🟢 Start 'deploy config file to S3' -----------------------------------+
| 
| 🚀️ deploy config file/files at s3://my-bucket-versioned/my-proje

### Read Config from S3 Backend

Now, you can use the Config.read() method to readthe config object from S3. If you want to read the all-in-one config object, then you could use ``parameter_name="${parameter_name}"``. If you want to read the config object of specific environment, you could use ``parameter="${parameter_name}-${env_name}"``.

On versioning enabled bucket, the ``config.version`` is the S3 object version_id that is managed by AWS.

In [26]:
config = Config.read(
    env_class=Env,
    env_enum_class=EnvEnum,
    bsm=bsm,
    parameter_name="my_project",
    s3folder_config=s3folder_config,
)
print("all in one config object:")
rprint(config)

all in one config object:


Config(
    data={
        '_shared': {'*.project_name': 'my_project'},
        'dev': {'username': 'dev.user.v1'},
        'prod': {'username': 'prod.user.v1'}
    },
    secret_data={'_shared': {}, 'dev': {'password': 'dev.password'}, 'prod': {'password': 'prod.password'}},
    Env=<class '__main__.Env'>,
    EnvEnum=<enum 'EnvEnum'>,
    version='cc4c8cf1-0e3f-4f41-9925-090769850fb6',
    _applied_data={
        'dev': {'username': 'dev.user.v1', 'project_name': 'my_project'},
        'prod': {'username': 'prod.user.v1', 'project_name': 'my_project'}
    },
    _applied_secret_data={'dev': {'password': 'dev.password'}, 'prod': {'password': 'prod.password'}},
    _merged={
        'dev': {'username': 'dev.user.v1', 'project_name': 'my_project', 'password': 'dev.password'},
        'prod': {'username': 'prod.user.v1', 'project_name': 'my_project', 'password': 'prod.password'}
    }
)

In [27]:
config = Config.read(
    env_class=Env,
    env_enum_class=EnvEnum,
    bsm=bsm,
    parameter_name="my_project-dev",
    s3folder_config=s3folder_config,
)
print("dev config object:")
rprint(config)

dev config object:


Config(
    data={'_shared': {'*.project_name': 'my_project'}, 'dev': {'username': 'dev.user.v1'}},
    secret_data={'_shared': {}, 'dev': {'password': 'dev.password'}},
    Env=<class '__main__.Env'>,
    EnvEnum=<enum 'EnvEnum'>,
    version='4ffd0b0a-0d41-4c7a-b2ca-c37edcfcf894',
    _applied_data={'dev': {'username': 'dev.user.v1', 'project_name': 'my_project'}},
    _applied_secret_data={'dev': {'password': 'dev.password'}},
    _merged={'dev': {'username': 'dev.user.v1', 'project_name': 'my_project', 'password': 'dev.password'}}
)

### Deploy a New Version of Config

When you deploy a new version of the config, it just creates a new version of the S3 object.

In [30]:
config_v2 = Config.read(
    env_class=Env,
    env_enum_class=EnvEnum,
    path_config=path_config_v2,
    path_secret_config=path_config_secret_v2,
)
rprint(config_v2)

Config(
    data={
        '_shared': {'*.project_name': 'my_project'},
        'dev': {'username': 'dev.user.v2'},
        'prod': {'username': 'prod.user.v2'}
    },
    secret_data={'_shared': {}, 'dev': {'password': 'dev.password'}, 'prod': {'password': 'prod.password'}},
    Env=<class '__main__.Env'>,
    EnvEnum=<enum 'EnvEnum'>,
    version='local',
    _applied_data={
        'dev': {'username': 'dev.user.v2', 'project_name': 'my_project'},
        'prod': {'username': 'prod.user.v2', 'project_name': 'my_project'}
    },
    _applied_secret_data={'dev': {'password': 'dev.password'}, 'prod': {'password': 'prod.password'}},
    _merged={
        'dev': {'username': 'dev.user.v2', 'project_name': 'my_project', 'password': 'dev.password'},
        'prod': {'username': 'prod.user.v2', 'project_name': 'my_project', 'password': 'prod.password'}
    }
)

In [31]:
deployment_list = config_v2.deploy(
    bsm=bsm, 
    s3folder_config=s3folder_config,
    # these two arguments are optional
    tags={"project_name": config_v2.project_name},
    verbose=True,
)

+----- ⏱ 🟢 Start 'deploy config file to S3' -----------------------------------+
| 
| 🚀️ deploy config file/files at s3://my-bucket-versioned/my-project/my_project.json ...
| preview at: https://console.aws.amazon.com/s3/object/my-bucket-versioned?prefix=my-project/my_project.json
| config data is the same as existing one, do nothing.
| 
+----- ⏰ 🟢 End 'deploy config file to S3', elapsed = 0.02 sec -----------------+
+----- ⏱ 🟢 Start 'deploy config file to S3' -----------------------------------+
| 
| 🚀️ deploy config file/files at s3://my-bucket-versioned/my-project/my_project-dev.json ...
| preview at: https://console.aws.amazon.com/s3/object/my-bucket-versioned?prefix=my-project/my_project-dev.json
| config data is the same as existing one, do nothing.
| 
+----- ⏰ 🟢 End 'deploy config file to S3', elapsed = 0.01 sec -----------------+
+----- ⏱ 🟢 Start 'deploy config file to S3' -----------------------------------+
| 
| 🚀️ deploy config file/files at s3://my-bucket-versioned/my-proje

You can see that each S3 object has two versions.

In [29]:
for s3path in S3Path(s3folder_config).iter_objects(bsm=bsm):
    for s3path_version in s3path.list_object_versions(bsm=bsm):
        print(s3path_version, s3path_version.version_id)

S3Path('s3://my-bucket-versioned/my-project/my_project-dev.json') 3dc70b16-9ccd-4d6b-9f6a-cf453d4d9c16
S3Path('s3://my-bucket-versioned/my-project/my_project-dev.json') b4a03b2d-d98b-4e39-a969-11b4add7384a
S3Path('s3://my-bucket-versioned/my-project/my_project-prod.json') 61574fd0-86c4-4fc1-bb79-36f5dc39c5c8
S3Path('s3://my-bucket-versioned/my-project/my_project-prod.json') 8051dec8-4907-4a89-b9d3-bcda3898168a
S3Path('s3://my-bucket-versioned/my-project/my_project.json') 59e3973f-c139-4f09-ab08-0b5ad045ffdf
S3Path('s3://my-bucket-versioned/my-project/my_project.json') d5f638d5-399b-4a90-ac9d-4dcb9dcf8635


You can read the config from S3 again, now it is the v2. And the config version matches the first version of ``s3://my-bucket-versioned/my-project/my_project.json``.

In [30]:
config = Config.read(
    env_class=Env,
    env_enum_class=EnvEnum,
    bsm=bsm,
    parameter_name="my_project",
    s3folder_config=s3folder_config,
)
print("all in one config object:")
rprint(config)

all in one config object:


Config(
    data={
        '_shared': {'*.project_name': 'my_project'},
        'dev': {'username': 'dev.user.v2'},
        'prod': {'username': 'prod.user.v2'}
    },
    secret_data={'_shared': {}, 'dev': {'password': 'dev.password'}, 'prod': {'password': 'prod.password'}},
    Env=<class '__main__.Env'>,
    EnvEnum=<enum 'EnvEnum'>,
    version='59e3973f-c139-4f09-ab08-0b5ad045ffdf',
    _applied_data={
        'dev': {'username': 'dev.user.v2', 'project_name': 'my_project'},
        'prod': {'username': 'prod.user.v2', 'project_name': 'my_project'}
    },
    _applied_secret_data={'dev': {'password': 'dev.password'}, 'prod': {'password': 'prod.password'}},
    _merged={
        'dev': {'username': 'dev.user.v2', 'project_name': 'my_project', 'password': 'dev.password'},
        'prod': {'username': 'prod.user.v2', 'project_name': 'my_project', 'password': 'prod.password'}
    }
)

### Delete and Clean Up

Normally, it is not necessary to delete any config deployments (S3 objects) as they have low cost and are critical to your application. If you accidentally deploy malformed config data, there is no need to delete it. Instead, you can simply create a new deployment with the corrected configuration.

To clean up all config objects in all environments, including the historical versions, you can retrieve the all-in-one object and then use the ``config.delete()`` method. By default, when you delete the object, only the latest version of S3 object is marked as deleted, making the config deployment "invisible (unaccessible)" to your application. However, the backup of all historical versions is retained. If you explicitly set ``include_history=True``, then all historical versions will be deleted as well. This design is implemented to prevent accidental deletion of all configurations. You can always recover the deleted version by removing the delete marker.

In [31]:
deployment_list = config_v2.delete(
    bsm=bsm, 
    s3folder_config=s3folder_config,
)

+----- ⏱ 🟢 Start 'delete config file from S3' ---------------------------------+
| 
| 🗑️ delete config file/files at: s3://my-bucket-versioned/my-project/my_project.json ...
| preview at: https://console.aws.amazon.com/s3/object/my-bucket-versioned?prefix=my-project/my_project.json
| done!
| 
+----- ⏰ 🟢 End 'delete config file from S3', elapsed = 0.01 sec ---------------+
+----- ⏱ 🟢 Start 'delete config file from S3' ---------------------------------+
| 
| 🗑️ delete config file/files at: s3://my-bucket-versioned/my-project/my_project-dev.json ...
| preview at: https://console.aws.amazon.com/s3/object/my-bucket-versioned?prefix=my-project/my_project-dev.json
| done!
| 
+----- ⏰ 🟢 End 'delete config file from S3', elapsed = 0.00 sec ---------------+
+----- ⏱ 🟢 Start 'delete config file from S3' ---------------------------------+
| 
| 🗑️ delete config file/files at: s3://my-bucket-versioned/my-project/my_project-prod.json ...
| preview at: https://console.aws.amazon.com/s3/object/my-bucke

In [32]:
deployment_list = config_v2.delete(
    bsm=bsm, 
    s3folder_config=s3folder_config,
    include_history=True,
)

+----- ⏱ 🟢 Start 'delete config file from S3' ---------------------------------+
| 
| 🗑️ delete config file/files at: s3://my-bucket-versioned/my-project/my_project.json ...
| preview at: https://console.aws.amazon.com/s3/object/my-bucket-versioned?prefix=my-project/my_project.json
| done!
| 
+----- ⏰ 🟢 End 'delete config file from S3', elapsed = 0.02 sec ---------------+
+----- ⏱ 🟢 Start 'delete config file from S3' ---------------------------------+
| 
| 🗑️ delete config file/files at: s3://my-bucket-versioned/my-project/my_project-dev.json ...
| preview at: https://console.aws.amazon.com/s3/object/my-bucket-versioned?prefix=my-project/my_project-dev.json
| done!
| 
+----- ⏰ 🟢 End 'delete config file from S3', elapsed = 0.00 sec ---------------+
+----- ⏱ 🟢 Start 'delete config file from S3' ---------------------------------+
| 
| 🗑️ delete config file/files at: s3://my-bucket-versioned/my-project/my_project-prod.json ...
| preview at: https://console.aws.amazon.com/s3/object/my-bucke